<a href="https://colab.research.google.com/github/azhar2205/paddle-ball-using-dqlearn/blob/master/kite_fno_data_viewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INPUT

In [ ]:
KITE_TOKEN = "enctoken duHop3kYhG14m9xB/S26YI+hJiWe22h1262G4xnw2upHR+8bsvZyipzy0l2GzKpRCLUZFCZ7ZvEx9xOHbDvKpfwSUsy4VyPUJNyBY9kMVzTlib6hzc7tIg=="

In [ ]:
PLOT_MODE = "DOWNLOAD_CHARTS" # "DOWNLOAD_CHARTS","DOWNLOAD_LIVE_CHARTS","PLOT_CHARTS","SHOW_CHARTS","OPT_LIVE","NIFTY_LIVE"
LIVE_CHARTS_FII_STRIKES = []

WK_OPT_EXPIRY_DATE = "2025-03-13"
# MN_OPT_EXPIRY_DATE = "2025-03-27"

In [ ]:
DD = 6
MM = 8
YYYY = 2024
START_HH = 9
START_MM = 15
END_HH = 15
END_MM = 30
OPT_EXPIRY_DATE = "2024-08-08"

In [ ]:
OPT_LIVE_STRIKE_MAP = {
    "21800_Put": "11469314",
    "21800_Call": "11468034",
    "21900_Put": "11479298",
    "21900_Call": "11474178",
    "22000_Put": "11524610",
    "22000_Call": "11523586",
}

NIFTY_INSTRUMENT_ID = "256265"

NIFTY_TOP10_INSTRUMENTS = {
    "HDFCBANK" : { "instrument_id" : "341249", "weightage" : 13.26 },
    "RELIANCE" : { "instrument_id" : "738561", "weightage" : 9.11 },
    "ICICIBANK" : { "instrument_id" : "1270529", "weightage" : 7.42 },
    "INFY" : { "instrument_id" : "408065", "weightage" : 5.89 },
    "ITC" : { "instrument_id" : "424961", "weightage" : 4.37 },
    "LT" : { "instrument_id" : "2939649", "weightage" : 4.26 },
    "TCS" : { "instrument_id" : "2953217", "weightage" : 4.05 },
    "AXISBANK" : { "instrument_id" : "1510401", "weightage" : 3.38 },
}

# Imports

In [ ]:
import calendar
import datetime
import itertools
import io
import json
import more_itertools
import numpy as np
import os
import pandas as pd
import pathlib
import re
import requests
from scipy.stats import norm
import time
import zoneinfo

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Configurations

In [ ]:
np.set_printoptions(formatter={"float_kind": "{:.4f}".format})
pd.options.display.float_format = "{:.4f}".format
pd.options.display.max_columns = None
pd.options.display.width = 1000

# Global Constants

In [ ]:
CHARTIQ_DATA_DIR = "/content/drive/MyDrive/nifty_kite_data"

INT = 10 / 100 # Risk-free Rate. Default 10%.
DIV = 0 / 100 # Annual dividend rate. Default zero.
VIX = 15.00 / 100 # Default volatility (if India VIX data is not available)

INTERPOLATION_FLAG = True

INSTRUMENTS_URL = "https://api.kite.trade/instruments"

In [ ]:
CHARTIQ_HEADERS = {
    "authority": "kite.zerodha.com",
    "accept": "*/*",
    "accept-language": "en-GB,en;q=0.9,en-US;q=0.8,mr;q=0.7,hi;q=0.6",
    "authorization": f"{KITE_TOKEN}",
    "referer": "https://kite.zerodha.com/static/build/chart-beta.html?v=3.3.2",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

# Global Variables

In [ ]:
def get_time_intervals():
    start_time = datetime.datetime(YYYY, MM, DD, START_HH, START_MM, 0)
    end_time = datetime.datetime(YYYY, MM, DD, END_HH, END_MM, 0)

    curr_time = start_time
    time_intervals = []
    while curr_time <= end_time:
        time_intervals.append(curr_time)
        curr_time += datetime.timedelta(minutes=1)

    return time_intervals

In [ ]:
spot_df = vix_df = None
oi_opt_df = vol_opt_df = price_opt_df = price_open_opt_df = price_high_opt_df = price_low_opt_df = None
tto_opt_df = nt_opt_df = iv_nr_df = prem_decay_df = None
oi_fut_df = vol_fut_df = price_fut_df = price_open_fut_df = price_high_fut_df = price_low_fut_df = None
vol_stk_df = price_close_stk_df = price_open_stk_df = price_high_stk_df = price_low_stk_df = None
CHARTIQ_DATA_TODAY_PATH = ""
NUM_DAYS_TO_EXPIRY = -1
TIME_INTERVALS = TIME_INTERVALS_STR = OPT_STRIKES = FUT_EXPIRIES = []
COLORS_STYLE = {}
COLOR_LIST = [
    "--k",
    "magenta", "magenta",
    "red", "red",
    "green", "green",
    "blue", "blue",
]
NIFTY_INSTRUMENTS_DF = pd.DataFrame([["NIFTY 50", f"{NIFTY_INSTRUMENT_ID}"]], columns=["name", "instrument_token"])


def set_global_variables():
    global CHARTIQ_DATA_TODAY_PATH
    global DD, MM, YYYY, END_HH, END_MM
    global NUM_DAYS_TO_EXPIRY
    global TIME_INTERVALS, TIME_INTERVALS_STR
    global COLORS_STYLE
    datetime_now = datetime.datetime.now(tz=zoneinfo.ZoneInfo("Asia/Kolkata"))
    if PLOT_MODE in ["DOWNLOAD_CHARTS", "DOWNLOAD_LIVE_CHARTS", "PLOT_CHARTS", "OPT_LIVE", "NIFTY_LIVE"]:
        DD = datetime_now.day
        MM = datetime_now.month
        YYYY = datetime_now.year
    if PLOT_MODE in ["OPT_LIVE", "NIFTY_LIVE"]:
        END_HH = datetime_now.hour
        END_MM = datetime_now.minute
    CHARTIQ_DATA_TODAY_PATH = f"{CHARTIQ_DATA_DIR}/{YYYY:04}/{MM:02}/{DD:02}"
    TIME_INTERVALS = get_time_intervals()
    if len(TIME_INTERVALS) > 601 and PLOT_MODE != "PLOT_CHARTS":
        raise RuntimeError(f"WARN: Too many time intervals = {len(TIME_INTERVALS)}.")
    TIME_INTERVALS_STR = [f"{t.hour:02}_{t.minute:02}" for t in TIME_INTERVALS]
    if PLOT_MODE not in ["DOWNLOAD_CHARTS", "DOWNLOAD_LIVE_CHARTS", "OPT_LIVE", "NIFTY_LIVE"]:
        NUM_DAYS_TO_EXPIRY = (datetime.datetime.strptime(OPT_EXPIRY_DATE, "%Y-%m-%d").date() - datetime.datetime(YYYY, MM, DD).date()).days + 1

    # COLORS_STYLE = {}
    # # COLOR_LIST = ["--k", "red", "gold", "blue", "lightcoral", "green", "darkorange", "magenta", "lime", "cyan", "yellow", "brown", "deepskyblue", "lightseagreen", "darkviolet", "olive"]
    # if len(COLOR_LIST) >= len(OPT_STRIKES):
    #     tab_colors = COLOR_LIST[:len(OPT_STRIKES)]
    #     for (s,c) in zip(OPT_STRIKES, tab_colors):
    #         COLORS_STYLE[s] = c
# end set_global_variables()


set_global_variables()

# Global Functions

In [ ]:
def interpolate_by_time(df, df_type):
    """
    df = columns->time, index->strike
    """
    if not INTERPOLATION_FLAG:
        print("'INTERPOLATION_FLAG' is OFF. Skipping interpolation.")
        return df

    df_arr = []
    for t in TIME_INTERVALS:
        hh_mm = f"{t.hour:02}_{t.minute:02}"
        if hh_mm not in df.columns:
            df_arr.append(pd.Series(data={}, dtype=float, index=df.index, name=hh_mm))
        else:
            df[hh_mm] = df[hh_mm].replace([np.inf, -np.inf, 0], np.nan)

    df_arr.append(df)
    df = pd.concat(df_arr, axis=1)
    df = df[df.columns.sort_values()]
    df = df.interpolate(method="linear", limit_direction="both", axis=1)
    df = df.astype(df_type)
    return df

In [ ]:
def interpolate_by_strike(df, df_type):
    """
    df = columns->time, index->strike
    """
    df = df.replace([np.inf, -np.inf, 0], np.nan)
    df = df.T[df.index.sort_values()].T
    df = df.interpolate(method="linear", limit_direction="both", axis=0)
    df = df.astype(df_type)
    return df

In [ ]:
def get_normalized_n_consolidate_call_n_put(df):
    df_sum = df.sum().sum()
    call_sr = df.T[[s for s in OPT_STRIKES if "00_Call" in s]].T.sum()
    put_sr = df.T[[s for s in OPT_STRIKES if "00_Put" in s]].T.sum()
    normalized_df = pd.concat([call_sr, put_sr], axis=1)
    normalized_df.columns = ["Call", "Put"]
    normalized_df = normalized_df * 200 / df_sum
    return normalized_df

In [ ]:
def get_premium_decay(spot_df, price_df):
    prem_decay_df = pd.DataFrame(columns=price_df.columns, index=price_df.index)
    for s in price_df.index:
        strike = int(s.split("_")[0])
        prem_decay_df.loc[s] = ((price_df.loc[s,] - (spot_df - strike)) * 100 / spot_df).values
        if s.split("_")[1] == "Put":
            prem_decay_df.loc[s] = prem_decay_df.loc[s] * -1
    return prem_decay_df

In [ ]:
def get_direction(row, money_flow_df):
    if money_flow_df.index.get_loc(row.name) == 0:
        return 0
    elif row["diff_avg_price"] > 0:
        return 1
    elif row["diff_avg_price"] < 0:
        return -1
    else:
        r = money_flow_df.index.get_loc(row.name) - 1
        c = money_flow_df.columns.get_loc("direction")
        return money_flow_df.iloc[r, c]
#
def compute_money_flow_df(strikes):
    money_flow_df_for_strikes = pd.DataFrame()
    for strike in strikes:
        money_flow_df = pd.DataFrame()
        money_flow_df["gross_turnover"] = tto_opt_df.T[strike]
        money_flow_df["gross_volume"] = vol_opt_df.T[strike]
        money_flow_df["prev_gross_turnover"] = money_flow_df["gross_turnover"].shift(periods=1, fill_value=0)
        money_flow_df["prev_gross_volume"] = money_flow_df["gross_volume"].shift(periods=1, fill_value=0)
        money_flow_df["diff_gross_turnover"] = money_flow_df["gross_turnover"] - money_flow_df["prev_gross_turnover"]
        money_flow_df["diff_gross_volume"] = money_flow_df["gross_volume"] - money_flow_df["prev_gross_volume"]
        money_flow_df["diff_gross_volume"] = money_flow_df["diff_gross_volume"].replace({0: 1})
        money_flow_df["avg_price"] = money_flow_df["diff_gross_turnover"] / money_flow_df["diff_gross_volume"]
        money_flow_df["prev_avg_price"] = money_flow_df["avg_price"].shift(periods=1, fill_value=0)
        money_flow_df["diff_avg_price"] = money_flow_df["avg_price"] - money_flow_df["prev_avg_price"]
        money_flow_df["direction"] = 0
        money_flow_df["direction"] = money_flow_df.apply(lambda row: get_direction(row, money_flow_df), axis=1)
        money_flow_df["delta_money_flow"] = abs(money_flow_df["diff_gross_turnover"]) * money_flow_df["direction"]
        money_flow_df["net_money_flow"] = money_flow_df["delta_money_flow"].cumsum()
        money_flow_df_for_strikes[strike] = money_flow_df["net_money_flow"]
    #
    return money_flow_df_for_strikes

In [ ]:
def get_direction_ohlcv(row, money_flow_df):
    if row["diff_avg_price"] > 0:
        return 1
    elif row["diff_avg_price"] < 0:
        return -1
    else:
        r = money_flow_df.index.get_loc(row.name) - 1
        c = money_flow_df.columns.get_loc("direction")
        return money_flow_df.iloc[r, c]
#
def compute_money_flow_ohlcv(instruments, ohlcv_map):
    o_df = ohlcv_map["O"]
    h_df = ohlcv_map["H"]
    l_df = ohlcv_map["L"]
    c_df = ohlcv_map["C"]
    v_df = ohlcv_map["V"]
    money_flow_df_for_instruments = pd.DataFrame()
    for instrument in instruments:
        money_flow_df = pd.DataFrame()
        money_flow_df["avg_price"] = (o_df.T[instrument] + h_df.T[instrument] + l_df.T[instrument] + c_df.T[instrument]) / 4
        money_flow_df["gross_turnover"] = money_flow_df["avg_price"] * v_df.T[instrument]
        money_flow_df["prev_avg_price"] = money_flow_df["avg_price"].shift(periods=1, fill_value=0)
        money_flow_df["diff_avg_price"] = money_flow_df["avg_price"] - money_flow_df["prev_avg_price"]
        money_flow_df["direction"] = 0
        # Compute direction for first record.
        money_flow_df.iloc[0, money_flow_df.columns.get_loc("direction")] = 1 if c_df.T[instrument][0] >= o_df.T[instrument][0] else -1
        # Compute direction for rest records.
        money_flow_df["direction"] = money_flow_df.apply(lambda row: get_direction_ohlcv(row, money_flow_df), axis=1)
        money_flow_df["delta_money_flow"] = abs(money_flow_df["gross_turnover"]) * money_flow_df["direction"]
        money_flow_df["net_money_flow"] = money_flow_df["delta_money_flow"].cumsum()
        money_flow_df_for_instruments[instrument] = money_flow_df["net_money_flow"]
    #
    return money_flow_df_for_instruments

In [ ]:
# https://quant.stackexchange.com/a/49724/55611

def d_one(s, k, t, rfr, sigma, div=0):
    d_1 = (np.log(s / k) + (rfr - div + sigma ** 2 / 2) * t) / (sigma * np.sqrt(t))
    return d_1


def d_two(s, k, t, rfr, sigma, div=0):
    d_2 = d_one(s, k, t, rfr, sigma, div) - sigma * np.sqrt(t)
    return d_2


def nd_one(right, s, k, t, rfr, sigma, div=0):
    if right == 'C':
        nd_1 = norm.cdf(d_one(s, k, t, rfr, sigma, div), 0, 1)
    elif right == 'P':
        nd_1 = norm.cdf(-d_one(s, k, t, rfr, sigma, div), 0, 1)
    return nd_1


def nd_two(right, s, k, t, rfr, sigma, div=0):
    if right == 'C':
        nd_2 = norm.cdf(d_two(s, k, t, rfr, sigma, div), 0, 1)
    elif right == 'P':
        nd_2 = norm.cdf(-d_two(s, k, t, rfr, sigma, div), 0, 1)
    return nd_2


def option_price(right, s, k, t, rfr, sigma, div=0):
    right = right.upper()
    t /= 365
    if right == 'C':
        price = (s * np.exp(-div * t) *
                 nd_one(right, s, k, t, rfr, sigma, div)
                 - k * np.exp(-rfr * t) *
                 nd_two(right, s, k, t, rfr, sigma, div))
    elif right == 'P':
        price = (k * np.exp(-rfr * t) *
                 nd_two(right, s, k, t, rfr, sigma, div)
                 - s * np.exp(-div * t) *
                 nd_one(right, s, k, t, rfr, sigma, div))
    return price


def option_vega(s, k, t, rfr, sigma, div=0):
    t /= 365
    vega = (.01 * s * np.exp(-div * t) * np.sqrt(t)
            * norm.pdf(d_one(s, k, t, rfr, sigma, div)))
    return vega


def implied_volatility_nr(right, s, k, t, rfr, price, div=0):
    epsilon = 0.000001
    sigma = 1.0

    def newton_raphson(right, s, k, t, rfr, sigma, price, epsilon, div=0):
        diff = np.abs(option_price(right, s, k, t, rfr, sigma, div) - price)
        while diff > epsilon:
            sigma = (sigma -
                     (option_price(right, s, k, t, rfr, sigma, div) - price) /
                     (option_vega(s, k, t, rfr, sigma, div) * 100))
            diff = np.abs(
                    option_price(right, s, k, t, rfr, sigma, div) - price)
        return sigma

    iv = newton_raphson(right, s, k, t, rfr, sigma, price, epsilon, div)
    return iv
#

ONE_UNIT = 0.1
STEP = 0.001

def implied_volatility_brute_force(right, s, k, t, rfr, strike_price, div=0, _sigma_initial_value=0.15):
        _sigma = _sigma_initial_value
        for i in range(200): #max number of calculations
            bs_price = option_price(right, s, k, t, rfr, sigma=_sigma, div=div)
            diff = strike_price - bs_price
            if diff > ONE_UNIT:
                _sigma = _sigma + STEP
            elif diff < 0 and abs(diff) > ONE_UNIT:
                _sigma = _sigma - STEP
            elif abs(diff) < ONE_UNIT:
                return _sigma
        return _sigma
#

# right = 'C' #'C' or 'P'
# s = 17672 #Spot Price
# k = 17600 #Strike
# t = 7 #Days to expiration
# rfr = .10 #Risk-free Rate
# sigma = .154 #VIX
# div = 0 #Annual dividend yield.
# call_premium = 208.65
# put_premium = 102.93

# option_price(right, s, k, t, rfr, sigma, div)

# implied_volatility_nr(right, s, k, t, rfr, call_premium, div)
# implied_volatility_brute_force(right, s, k, t, rfr, call_premium, div)

# implied_volatility_nr("C", 17616.3, 17300, 1, 0.1, 231.15, 0.0)
# implied_volatility_brute_force("C", 17616.3, 17300, 1, 0.1, 231.15, 0)

In [ ]:
def compute_premium_using_bs(spot_df, price_df, vix_df):
    prem_bs_df = pd.DataFrame(columns=spot_df.columns, index=price_df.index)
    for s in price_df.index:
        strike = int(s.split("_")[0])
        right = s.split("_")[1][0] # "C" or "P"
        for t in spot_df.columns:
            spot_price = spot_df.loc["NIFTY", t]
            vix_t = VIX
            if len(vix_df):
                vix_t = vix_df.loc["VIX", t] / 100
            premium = option_price(right, spot_price, strike, NUM_DAYS_TO_EXPIRY, INT, vix_t, DIV)
            prem_bs_df.loc[s, t] = premium

    return prem_bs_df

In [ ]:
def compute_iv_using_nr(spot_df, price_df):
    iv_nr_df = pd.DataFrame(columns=spot_df.columns, index=price_df.index)
    for s in price_df.index:
        strike = int(s.split("_")[0])
        right = s.split("_")[1][0] # "C" or "P"
        for t in spot_df.columns:
            strike_price = price_df.loc[s, t]
            spot_price = spot_df.loc["NIFTY", t]
            iv = implied_volatility_nr(right, spot_price, strike, NUM_DAYS_TO_EXPIRY, INT, strike_price, DIV)
            iv_nr_df.loc[s, t] = iv

    iv_nr_df = interpolate_by_strike(iv_nr_df, float)
    return iv_nr_df

In [ ]:
def compute_iv_using_bf(spot_df, price_df, vix_df):
    iv_bf_df = pd.DataFrame(columns=spot_df.columns, index=price_df.index)
    for s in price_df.index:
        strike = int(s.split("_")[0])
        right = s.split("_")[1][0] # "C" or "P"
        for t in spot_df.columns:
            strike_price = price_df.loc[s, t]
            spot_price = spot_df.loc["NIFTY", t]
            vix_t = VIX
            if len(vix_df):
                vix_t = vix_df.loc["VIX", t] / 100
            iv = implied_volatility_brute_force(right, spot_price, strike, NUM_DAYS_TO_EXPIRY, INT, strike_price, DIV, vix_t)
            if iv < 0.001:
                iv = np.nan
            iv_bf_df.loc[s, t] = iv

    iv_bf_df = interpolate_by_strike(iv_bf_df, float)
    return iv_bf_df

##Plot

In [ ]:
def plot_df(df, plot_spot=False, figsize=None, ax=None, title=None, style=None, kind="line"):
    if title:
        ax.set_title(title, pad=0)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ",")))
    ax.margins(x=0)
    ax.grid(True, linestyle="--", linewidth=.5)
    for spine in ax.spines.values():
        spine.set_visible(False)

    ticks_df = pd.DataFrame(data=[t for t in df.index], columns=["ticks"])
    ticks_df["ticks"] = ticks_df["ticks"].apply(lambda t: t if t.endswith("_00") or t.endswith("_15") or t.endswith("_30") or t.endswith("_45") else None)
    ticks_df = ticks_df.dropna(subset=["ticks"])
    tick_indices = ticks_df.index.values
    tick_labels = ticks_df.ticks.values
    ax.xaxis.set_ticks(ticks=tick_indices, labels=tick_labels)

    if (len(df.columns) == 1) or (len(df.columns) == 2 and "NIFTY" in df.columns):
        strike = df.columns[0]
        color = style[strike]
        if kind == "bar":
            ax.bar(x=df.index.values, height=df[strike].values, color=color, label=strike)
        else:
            ax.plot(df.index.values, df[strike].values, color=color, label=strike)
        ax.legend()
    else:
        y_df = df[[strike for strike in df.columns if strike != "NIFTY"]]
        ax.plot(y_df.index.values, y_df.values)
        ax.legend(y_df.columns)

    if plot_spot:
        ax2 = ax.twinx()
        ax2.plot(df.index.values, df["NIFTY"].values, color="black", linestyle="dashed", label="NIFTY")
#

# fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(20, 8))
# style = {"21500_Call": "green"}
# plot_df(vol_opt_df.T[["21500_Call"]], ax=axs[0], title="VOL", style=style, kind="bar")
# plot_df(vol_opt_df.T[["21500_Call"]], ax=axs[1], title="VOL", style=style, kind="line")

In [ ]:
def plot_x_roc(x_df, scale=False, include_strikes=[], exclude_strikes=[], plot_spot=False, figsize=None, ax=None, title=None, style=None, kind="line"):
    x_diff_df = pd.DataFrame(columns=sorted(x_df.columns))
    x_diff_df[x_df.columns[0]] = x_df[x_diff_df.columns[0]]
    for t in x_diff_df.columns[1:]:
        x_diff_df[t] = x_df[t] - x_df[x_df.columns[0]]

    if include_strikes:
        x_diff_df = x_diff_df[x_diff_df.index.isin(include_strikes)]

    if exclude_strikes:
        x_diff_df = x_diff_df[~x_diff_df.index.isin(exclude_strikes)]

    x_diff_df = x_diff_df.T

    if plot_spot:
        x_diff_df = x_diff_df.join(spot_df.T)

    if scale:
        for s in sorted(x_diff_df.columns):
            max_val = x_diff_df[s].max()
            x_diff_df.loc[:,s] = x_diff_df[s] * 100 / max_val

    x_diff_df = x_diff_df.iloc[1:,]

    x_diff_df = x_diff_df[sorted(x_diff_df.columns)]
    plot_df(x_diff_df, plot_spot=plot_spot, figsize=figsize, ax=ax, title=title, style=style, kind=kind)

In [ ]:
def plot_x_diff(x_df, scale=False, include_strikes=[], exclude_strikes=[], plot_spot=False, plot_spot_diff=False, figsize=None, ax=None, title=None, style=None, kind="line"):
    if plot_spot and plot_spot_diff:
        raise ValueError(f"Either paramter should be used (not both): 'plot_spot', 'plot_spot_diff'")

    x_diff_df = pd.DataFrame(columns=sorted(x_df.columns[1:]))
    for t1, t2 in more_itertools.pairwise(sorted(x_df.columns)):
        x_diff_df[t2] = x_df[t2] - x_df[t1]

    if include_strikes:
        x_diff_df = x_diff_df[x_diff_df.index.isin(include_strikes)]

    if exclude_strikes:
        x_diff_df = x_diff_df[~x_diff_df.index.isin(exclude_strikes)]

    x_diff_df = x_diff_df.T

    if plot_spot:
        x_diff_df = x_diff_df.join(spot_df.T)
    elif plot_spot_diff:
        spot_diff_df = pd.DataFrame(columns=sorted(spot_df.columns[1:]))
        for t1, t2 in more_itertools.pairwise(sorted(spot_df.columns)):
            spot_diff_df[t2] = spot_df[t2] - spot_df[t1]
        x_diff_df = x_diff_df.join(spot_diff_df.T)

    if scale:
        for s in sorted(x_diff_df.columns):
            max_val = x_diff_df[s].max()
            x_diff_df.loc[:,s] = x_diff_df[s] * 100 / max_val

    x_diff_df = x_diff_df[sorted(x_diff_df.columns)]
    plot_df(x_diff_df, plot_spot=(plot_spot or plot_spot_diff), figsize=figsize, ax=ax, title=title, style=style, kind=kind)

##Data Load

In [ ]:
def read_chartiq_idx_fno_file(fno_filepath):
    fno_df = pd.read_csv(fno_filepath)
    return process_idx_fno_chartiq_df(fno_df)
#

def process_idx_fno_chartiq_df(df):
    fno_df = df.copy()
    fno_df["Date"] = fno_df["Date"].apply(parse_datetime)
    fno_df = filter_exta_data(fno_df)
    fno_df = fno_df.drop_duplicates(subset=["Date"])
    fno_df["Date"] = fno_df["Date"].apply(convert_timestamp_to_hhmm)
    fno_df = fno_df.set_index("Date")
    fno_df.index.names = [""]
    return fno_df
#

def parse_datetime(ts_in):
    if PLOT_MODE in ["DOWNLOAD_CHARTS", "DOWNLOAD_LIVE_CHARTS", "OPT_LIVE", "NIFTY_LIVE"]:
        dt = datetime.datetime.strptime(ts_in, "%Y-%m-%dT%H:%M:%S%z")
    else:
        ts = ts_in.replace("GMT+0530 (India Standard Time)", "+0530")
        try:
            dt = datetime.datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S%z")
        except ValueError:
            dt = datetime.datetime.strptime(ts, "%a %b %d %Y %H:%M:%S %z")
    return dt
#

def convert_timestamp_to_hhmm(dt):
    return f"{dt.hour:02}_{dt.minute:02}"
#

def filter_exta_data(df_in):
    start_time = datetime.datetime(YYYY, MM, DD, START_HH, START_MM, 0, tzinfo=zoneinfo.ZoneInfo("Asia/Kolkata"))
    end_time = datetime.datetime(YYYY, MM, DD, END_HH, END_MM, 0, tzinfo=zoneinfo.ZoneInfo("Asia/Kolkata"))
    df_out = df_in[(df_in["Date"] >= start_time) & (df_in["Date"] <= end_time)]
    return df_out
#

def build_attr_df_from_df_dict(attr, df_dict):
    sr_arr = []
    for k, k_df in df_dict.items():
        sr_arr.append(pd.Series(data=k_df[attr], name=k))
    return pd.concat(sr_arr, axis=1)

In [ ]:
def parse_index_data(dirpath):
    index_files = [os.path.join(dirpath, f) for f in os.listdir(dirpath) if "NIFTY 50 " in f or "INDIA VIX " in f or f == "NIFTY.csv" or f == "VIX.csv"]
    df_dict = {}
    for f in index_files:
        index_name = pathlib.Path(f).stem
        index_key = "VIX" if "VIX" in index_name else "NIFTY"
        df_dict[index_key] = read_chartiq_idx_fno_file(f)
    return df_dict
#

def parse_options_data(dirpath):
    global OPT_STRIKES
    options_files = [os.path.join(dirpath, f) for f in os.listdir(dirpath) if " CE " in f or " PE " in f or "_Call" in f or "_Put" in f]
    df_dict = {}
    OPT_STRIKES = []
    for f in options_files:
        strike = pathlib.Path(f).stem
        if " CE " in f or " PE " in f:
            strike = re.search(r"\d+\s(CE|PE)", strike).group()
            strike = strike.replace(" CE", "_Call").replace(" PE", "_Put")
        df_dict[strike] = read_chartiq_idx_fno_file(f)
        OPT_STRIKES.append(strike)
    return df_dict
#

def parse_futures_data(dirpath):
    global FUT_EXPIRIES
    futures_files = [os.path.join(dirpath, f) for f in os.listdir(dirpath) if " FUT " in f or "_Future" in f]
    df_dict = {}
    FUT_EXPIRIES = []
    for f in futures_files:
        expiry = pathlib.Path(f).stem
        if " FUT " in f:
            expiry = re.search(r"[A-Za-z]+ FUT", expiry).group()
            expiry = expiry.replace(" FUT", "_Future")
        df_dict[expiry] = read_chartiq_idx_fno_file(f)
        FUT_EXPIRIES.append(expiry)
    return df_dict
#

def get_instruments_historical_url(instrument_id, oi=0):
    return f"https://kite.zerodha.com/oms/instruments/historical/{instrument_id}/minute?user_id=JV5904&oi={oi}&from={YYYY}-{MM:02}-{DD:02}&to={YYYY}-{MM:02}-{DD:02}"
#

def download_index_data(instruments_df):
    df_dict = {}
    for idx, row in instruments_df.iterrows():
        index_name = row["name"]
        instrument_id = row["instrument_token"]
        CHARTIQ_URL = get_instruments_historical_url(instrument_id, oi=0)
        idx_data = call_chartiq_api(CHARTIQ_URL, retry=True)
        idx_data_df = pd.DataFrame(idx_data, columns=["Date", "Open", "High", "Low", "Close", "Volume"])
        idx_data_df = idx_data_df.drop(columns="Volume")
        index_key = "NIFTY" if "NIFTY 50" in index_name else "VIX"
        if PLOT_MODE in ["DOWNLOAD_CHARTS", "DOWNLOAD_LIVE_CHARTS"]:
            idx_data_df.to_csv(f"{CHARTIQ_DATA_TODAY_PATH}/{index_key}.csv", index=False)
        df_dict[index_key] = process_idx_fno_chartiq_df(idx_data_df)
        time.sleep(1)
    return df_dict
#

def download_options_data(instruments_df):
    global OPT_STRIKES
    df_dict = {}
    OPT_STRIKES = []
    for idx, row in instruments_df.iterrows():
        strike = row["strike"]
        instrument_id = row["instrument_token"]
        CHARTIQ_URL = get_instruments_historical_url(instrument_id, oi=1)
        opt_data = call_chartiq_api(CHARTIQ_URL, retry=True)
        opt_data_df = pd.DataFrame(opt_data, columns=["Date", "Open", "High", "Low", "Close", "Volume", "OI"])
        opt_data_df.to_csv(f"{CHARTIQ_DATA_TODAY_PATH}/{strike}.csv", index=False)
        df_dict[strike] = process_idx_fno_chartiq_df(opt_data_df)
        OPT_STRIKES.append(strike)
        time.sleep(1)
    return df_dict
#

def download_futures_data(instruments_df):
    global FUT_EXPIRIES
    df_dict = {}
    FUT_EXPIRIES = []
    for idx, row in instruments_df.iterrows():
        expiry = row["expiry"]
        instrument_id = row["instrument_token"]
        CHARTIQ_URL = get_instruments_historical_url(instrument_id, oi=1)
        fut_data = call_chartiq_api(CHARTIQ_URL, retry=True)
        fut_data_df = pd.DataFrame(fut_data, columns=["Date", "Open", "High", "Low", "Close", "Volume", "OI"])
        fut_data_df.to_csv(f"{CHARTIQ_DATA_TODAY_PATH}/{expiry}.csv", index=False)
        df_dict[expiry] = process_idx_fno_chartiq_df(fut_data_df)
        FUT_EXPIRIES.append(expiry)
        time.sleep(1)
    return df_dict
#

def download_options_live_data():
    global OPT_STRIKES
    df_dict = {}
    OPT_STRIKES = []
    for strike, instrument_id in OPT_LIVE_STRIKE_MAP.items():
        CHARTIQ_URL = get_instruments_historical_url(instrument_id, oi=1)
        opt_data = call_chartiq_api(CHARTIQ_URL)
        opt_data_df = pd.DataFrame(opt_data, columns=["Date", "Open", "High", "Low", "Close", "Volume", "OI"])
        df_dict[strike] = process_idx_fno_chartiq_df(opt_data_df)
        OPT_STRIKES.append(strike)
        time.sleep(1)
    return df_dict
#

def download_nifty_top10_live_data():
    df_dict = {}
    for symbol, instrument_map in NIFTY_TOP10_INSTRUMENTS.items():
        CHARTIQ_URL = get_instruments_historical_url(instrument_map["instrument_id"], oi=0)
        stk_data = call_chartiq_api(CHARTIQ_URL)
        stk_data_df = pd.DataFrame(stk_data, columns=["Date", "Open", "High", "Low", "Close", "Volume"])
        df_dict[symbol] = process_idx_fno_chartiq_df(stk_data_df)
        time.sleep(1)
    return df_dict
#

In [ ]:
def build_fno_stk_df(df_dict, oi=True):
    # Build Open Interest df
    oi_fno_df = None
    if oi:
        oi_fno_df = build_attr_df_from_df_dict("OI", df_dict)
        oi_fno_df = oi_fno_df.T
        oi_fno_df = oi_fno_df[sorted(oi_fno_df.columns)]
        oi_fno_df = interpolate_by_time(oi_fno_df, int)
        oi_fno_df = oi_fno_df.astype(int)
        oi_fno_df = oi_fno_df[sorted(TIME_INTERVALS_STR)]
        oi_fno_df = oi_fno_df.copy()

    # Build Volume df
    vol_fno_df = build_attr_df_from_df_dict("Volume", df_dict)
    vol_fno_df = vol_fno_df.T
    vol_fno_df = vol_fno_df[sorted(vol_fno_df.columns)]
    vol_fno_df = interpolate_by_time(vol_fno_df, int)
    vol_fno_df = vol_fno_df[sorted(vol_fno_df.columns)]
    vol_fno_df = vol_fno_df.T.rolling(window=1, min_periods=1).sum().T
    vol_fno_df = vol_fno_df.astype(int)
    vol_fno_df = vol_fno_df[sorted(TIME_INTERVALS_STR)]
    vol_fno_df = vol_fno_df.copy()

    # Build Close Price df
    price_fno_df = build_attr_df_from_df_dict("Close", df_dict)
    price_fno_df = price_fno_df.T
    price_fno_df = price_fno_df[sorted(price_fno_df.columns)]
    price_fno_df = interpolate_by_time(price_fno_df, float)
    price_fno_df = price_fno_df[sorted(TIME_INTERVALS_STR)]
    price_fno_df = price_fno_df.copy()

    # Build Open Price df
    price_open_fno_df = build_attr_df_from_df_dict("Open", df_dict)
    price_open_fno_df = price_open_fno_df.T
    price_open_fno_df = price_open_fno_df[sorted(price_open_fno_df.columns)]
    price_open_fno_df = interpolate_by_time(price_open_fno_df, float)
    price_open_fno_df = price_open_fno_df[sorted(TIME_INTERVALS_STR)]
    price_open_fno_df = price_open_fno_df.copy()

    # Build High Price df
    price_high_fno_df = build_attr_df_from_df_dict("High", df_dict)
    price_high_fno_df = price_high_fno_df.T
    price_high_fno_df = price_high_fno_df[sorted(price_high_fno_df.columns)]
    price_high_fno_df = interpolate_by_time(price_high_fno_df, float)
    price_high_fno_df = price_high_fno_df[sorted(TIME_INTERVALS_STR)]
    price_high_fno_df = price_high_fno_df.copy()

    # Build Low Price df
    price_low_fno_df = build_attr_df_from_df_dict("Low", df_dict)
    price_low_fno_df = price_low_fno_df.T
    price_low_fno_df = price_low_fno_df[sorted(price_low_fno_df.columns)]
    price_low_fno_df = interpolate_by_time(price_low_fno_df, float)
    price_low_fno_df = price_low_fno_df[sorted(TIME_INTERVALS_STR)]
    price_low_fno_df = price_low_fno_df.copy()

    return oi_fno_df, vol_fno_df, price_fno_df, price_open_fno_df, price_high_fno_df, price_low_fno_df

In [ ]:
def build_idx_df(df_dict):
    attr_df = build_attr_df_from_df_dict("Close", df_dict)

    # Build SPOT df
    spot_df = attr_df[["NIFTY"]].copy()
    spot_df = spot_df.T
    spot_df = spot_df[sorted(spot_df.columns)]
    spot_df = interpolate_by_time(spot_df, float)
    spot_df = spot_df[sorted(TIME_INTERVALS_STR)]
    spot_df = spot_df.copy()

    # Build VIX df
    vix_df = None
    if PLOT_MODE not in ["DOWNLOAD_LIVE_CHARTS", "OPT_LIVE", "NIFTY_LIVE"]:
        vix_df = attr_df[["VIX"]]
        vix_df = vix_df.T
        vix_df = vix_df[sorted(vix_df.columns)]
        vix_df = interpolate_by_time(vix_df, float)
        vix_df = vix_df[sorted(TIME_INTERVALS_STR)]
        vix_df = vix_df.copy()

    return spot_df, vix_df

In [ ]:
def call_chartiq_api(url, retry=False):
    try:
        response = requests.get(url, headers=CHARTIQ_HEADERS, timeout=10)
        if not response.ok:
            print(f"ERROR: {url} failed with status code {response.status_code} and body {response.text}")
            response.raise_for_status()
        response_json = json.loads(response.text)
        if response_json["status"] != "success":
            raise ValueError(f"{url} failed")
        return response_json["data"]["candles"]
    except (requests.ReadTimeout, json.JSONDecodeError) as e:
        if retry:
            print("Retrying after 5 sec...")
            time.sleep(5)
            return call_chartiq_api(url, retry=False)
        else:
            raise e
#

def call_instruments_api(retry=False):
    try:
        response = requests.get(INSTRUMENTS_URL, timeout=10)
        if not response.ok:
            print(f"ERROR: {INSTRUMENTS_URL} failed with status code {response.status_code} and body {response.text}")
            response.raise_for_status()
        instruments_df = pd.read_csv(io.StringIO(response.text), header=0)
        # README: In case of failure, download the file using command in cell below.
        # instruments_df = pd.read_csv('/content/instruments.csv', header=0)
        return instruments_df
    except (requests.ReadTimeout, json.JSONDecodeError) as e:
        if retry:
            print("Retrying after 5 sec...")
            time.sleep(5)
            return call_instruments_api(retry=False)
        else:
            raise e
#

In [ ]:
# README: In case of failure, download the file using this command.
# !wget https://api.kite.trade/instruments -O /content/instruments.csv

# Analysis

## Data Load

In [ ]:
def download_n_save_data():
    global spot_df, vix_df, \
    oi_opt_df, vol_opt_df, price_opt_df, price_open_opt_df, price_high_opt_df, price_low_opt_df, \
    tto_opt_df, nt_opt_df, iv_nr_df, prem_decay_df, \
    oi_fut_df, vol_fut_df, price_fut_df, price_open_fut_df, price_high_fut_df, price_low_fut_df

    exist_ok = False
    if PLOT_MODE in ["DOWNLOAD_LIVE_CHARTS"]:
        exist_ok=True
    pathlib.Path(CHARTIQ_DATA_TODAY_PATH).mkdir(parents=True, exist_ok=exist_ok)

    instruments_df = call_instruments_api(retry=True)

    # set_current_week_option_expiry_date(instruments_df)

    # Download, save, and parse Index data from API
    index_instruments = ["NIFTY 50", "INDIA VIX"]
    if PLOT_MODE in ["DOWNLOAD_LIVE_CHARTS"]:
        index_instruments = ["NIFTY 50"]
    idx_instruments_df = instruments_df[(instruments_df["exchange"] == "NSE") & (instruments_df["segment"] == "INDICES") & (instruments_df["name"].isin(index_instruments))][["name", "instrument_token"]]
    idx_df_map = download_index_data(idx_instruments_df)
    spot_df, vix_df = build_idx_df(idx_df_map)

    opt_instruments_df, fut_instruments_df = get_fno_instruments(instruments_df, spot_df)

    # Options data
    opt_df_map = download_options_data(opt_instruments_df)
    oi_opt_df, vol_opt_df, price_opt_df, price_open_opt_df, price_high_opt_df, price_low_opt_df = build_fno_stk_df(opt_df_map)
    tto_opt_df = vol_opt_df * (price_open_opt_df + price_high_opt_df + price_low_opt_df + price_opt_df) / 4

    # Futures data
    if not PLOT_MODE in ["DOWNLOAD_LIVE_CHARTS"]:
        fut_df_map = download_futures_data(fut_instruments_df)
        oi_fut_df, vol_fut_df, price_fut_df, price_open_fut_df, price_high_fut_df, price_low_fut_df = build_fno_stk_df(fut_df_map)
        tto_fut_df = vol_fut_df * (price_open_fut_df + price_high_fut_df + price_low_fut_df + price_fut_df) / 4
#

def set_current_week_option_expiry_date(instruments_df):
    opt_expiry_date_df = instruments_df[
        (instruments_df["segment"]=="NFO-OPT")
        & (instruments_df["name"]=="NIFTY")
    ][["expiry"]]
    opt_expiry_date_df = opt_expiry_date_df.drop_duplicates()
    opt_expiry_date_df["days_to_expiry"] = opt_expiry_date_df.apply(lambda x: (datetime.datetime.strptime(x.expiry, "%Y-%m-%d").date() - datetime.datetime(YYYY, MM, DD).date()).days + 1, axis=1)
    opt_expiry_date_df = opt_expiry_date_df[(opt_expiry_date_df["days_to_expiry"] >= 1) & (opt_expiry_date_df["days_to_expiry"] <= 7)]
    opt_expiry_date_df = opt_expiry_date_df.sort_values("expiry", ascending=True)[:1]
    global OPT_EXPIRY_DATE
    OPT_EXPIRY_DATE = opt_expiry_date_df["expiry"].values[0]
#

def get_fno_instruments(instruments_df, spot_df):
    min_strike = int(spot_df.T["NIFTY"].min() / 100) * 100
    max_strike = int((spot_df.T["NIFTY"].max() / 100) + 1) * 100
    ce_strikes = [s for s in range(min_strike - 700, max_strike + 800, 100)]
    pe_strikes = [s for s in range(min_strike - 700, max_strike + 800, 100)]
    if PLOT_MODE == "DOWNLOAD_LIVE_CHARTS":
        ce_strikes = [s for s in range(min_strike - 200, max_strike + 100, 100)] + [int(s.replace("_Call", "")) for s in LIVE_CHARTS_FII_STRIKES if "00_Call" in s]
        pe_strikes = [s for s in range(min_strike - 100, max_strike + 200, 100)] + [int(s.replace("_Put", "")) for s in LIVE_CHARTS_FII_STRIKES if "00_Put" in s]

    ce_instruments_df = instruments_df[
        (instruments_df["exchange"]=="NFO")
        & (instruments_df["instrument_type"]=="CE")
        & (instruments_df["name"]=="NIFTY")
        & (instruments_df["expiry"]==OPT_EXPIRY_DATE)
        & (instruments_df["strike"].isin(ce_strikes))
    ][["strike", "instrument_token"]]
    ce_instruments_df["strike"] = ce_instruments_df.apply(lambda x: str(x.strike.astype(int)) + "_Call", axis=1)

    pe_instruments_df = instruments_df[
        (instruments_df["exchange"]=="NFO")
        & (instruments_df["instrument_type"]=="PE")
        & (instruments_df["name"]=="NIFTY")
        & (instruments_df["expiry"]==OPT_EXPIRY_DATE)
        & (instruments_df["strike"].isin(pe_strikes))
    ][["strike", "instrument_token"]]
    pe_instruments_df["strike"] = pe_instruments_df.apply(lambda x: str(x.strike.astype(int)) + "_Put", axis=1)

    opt_instruments_df = pd.concat([pe_instruments_df, ce_instruments_df])

    fut_instruments_df = instruments_df[
        (instruments_df["exchange"]=="NFO")
        & (instruments_df["instrument_type"]=="FUT")
        & (instruments_df["name"]=="NIFTY")
    ][["expiry", "instrument_token"]]
    fut_instruments_df["expiry"] = fut_instruments_df.apply(lambda x: calendar.month_abbr[datetime.datetime.strptime(x.expiry, "%Y-%m-%d").date().month].upper() + "_Future", axis=1)
    if PLOT_MODE == "DOWNLOAD_LIVE_CHARTS":
        fut_instruments_df = pd.DataFrame()

    return opt_instruments_df, fut_instruments_df

In [ ]:
def build_dataframes():
    global spot_df, vix_df, \
    oi_opt_df, vol_opt_df, price_opt_df, price_open_opt_df, price_high_opt_df, price_low_opt_df, \
    tto_opt_df, nt_opt_df, iv_nr_df, prem_decay_df, \
    oi_fut_df, vol_fut_df, price_fut_df, price_open_fut_df, price_high_fut_df, price_low_fut_df

    # Parse index data from files
    idx_df_map = parse_index_data(CHARTIQ_DATA_TODAY_PATH)
    spot_df, vix_df = build_idx_df(idx_df_map)

    # Parse options data from files
    opt_df_map = parse_options_data(CHARTIQ_DATA_TODAY_PATH)
    oi_opt_df, vol_opt_df, price_opt_df, price_open_opt_df, price_high_opt_df, price_low_opt_df = build_fno_stk_df(opt_df_map)
    tto_opt_df = vol_opt_df * (price_open_opt_df + price_high_opt_df + price_low_opt_df + price_opt_df) / 4
    # nt_opt_df = (vol_opt_df * 4 / (price_open_opt_df + price_high_opt_df + price_low_opt_df + price_opt_df)).astype(int)
    # iv_nr_df = compute_iv_using_bf(spot_df=spot_df, price_df=price_opt_df, vix_df=vix_df) * 100
    # Use prem decay only for relative comparison among diff strikes of Call and Put. Not to be used for single strike.
    prem_decay_df = get_premium_decay(spot_df=spot_df, price_df=price_opt_df)

    # Parse futures data from files
    fut_df_map = parse_futures_data(CHARTIQ_DATA_TODAY_PATH)
    oi_fut_df, vol_fut_df, price_fut_df, price_open_fut_df, price_high_fut_df, price_low_fut_df = build_fno_stk_df(fut_df_map)
    tto_fut_df = vol_fut_df * (price_open_fut_df + price_high_fut_df + price_low_fut_df + price_fut_df) / 4

## Plot Strikes

In [ ]:
def plot_strikes(strikes, save_filepath=None):
    ohlcv_map = {
        "O" : price_open_opt_df,
        "H" : price_high_opt_df,
        "L" : price_low_opt_df,
        "C" : price_opt_df,
        "V" : vol_opt_df,
    }
    nrows = 3
    ncols = len(strikes)
    if END_HH - START_HH >= 5:
        w = 20 * ncols
        h = 8 * nrows
    else:
        w = 10 * ncols
        h = 6 * nrows
    fig, axs = plt.subplots(nrows=nrows, ncols=len(strikes), figsize=(w, h))
    if len(strikes) == 1:
        axs = axs.reshape(len(axs),1)
    for s in range(0, len(strikes)):
        r = 0
        style = {**{s:"red" if "_Call" in s else "green" for s in strikes}, "NIFTY": "black"}
        plot_x_diff(oi_opt_df, include_strikes=[strikes[s]], ax=axs[r, s], title="OI", style=style, kind="bar")
        r = r + 1
        plot_df(vol_opt_df.T[[strikes[s]]], ax=axs[r, s], title="VOL", style=style, kind="bar")
        r = r + 1
        plot_x_roc(price_opt_df, include_strikes=[strikes[s]], ax=axs[r, s], title="Premium", style=style)
        # r = r + 1
        # plot_x_roc(iv_nr_df, include_strikes=[strikes[s]], ax=axs[r, s], title="IV by NR", style=style)
        # r = r + 1
        # plot_df(compute_money_flow_ohlcv([strikes[s]], ohlcv_map), ax=axs[r, s], title="Money Flow", style=style)
        # r = r + 1
    if save_filepath:
        plt.savefig(save_filepath, bbox_inches="tight")
        plt.close(fig)
#

def plot_futures(expiries, save_filepath=None):
    ohlcv_map = {
        "O" : price_open_fut_df,
        "H" : price_high_fut_df,
        "L" : price_low_fut_df,
        "C" : price_fut_df,
        "V" : vol_fut_df,
    }
    nrows = 3
    ncols = len(expiries)
    if END_HH - START_HH >= 5:
        w = 20 * ncols
        h = 8 * nrows
    else:
        w = 10 * ncols
        h = 6 * nrows
    fig, axs = plt.subplots(nrows=nrows, ncols=len(expiries), figsize=(w, h))
    if len(expiries) == 1:
        axs = axs.reshape(len(axs),1)
    for s in range(0, len(expiries)):
        r = 0
        style = {**{e:"blue" for e in expiries}, "NIFTY": "black"}
        plot_x_diff(oi_fut_df, include_strikes=[expiries[s]], ax=axs[r, s], title="OI", style=style, kind="bar")
        r = r + 1
        plot_df(vol_fut_df.T[[expiries[s]]], ax=axs[r, s], title="VOL", style=style, kind="bar")
        r = r + 1
        plot_x_roc(price_fut_df, include_strikes=[expiries[s]], ax=axs[r, s], title="Premium", style=style, plot_spot=True)
        # r = r + 1
        # plot_df(compute_money_flow_ohlcv([expiries[s]], ohlcv_map), ax=axs[r, s], title="Money Flow", style=style)
        # r = r + 1
    if save_filepath:
        plt.savefig(save_filepath, bbox_inches="tight")
        plt.close(fig)
#

def plot_individual_strikes(strikes, save_filepath=None):
    ohlcv_map = {
        "O" : price_open_opt_df,
        "H" : price_high_opt_df,
        "L" : price_low_opt_df,
        "C" : price_opt_df,
        "V" : vol_opt_df,
    }
    nrows = 3
    w = 30
    h = 18
    for s in range(0, len(strikes)):
        fig, axs = plt.subplots(nrows=nrows, ncols=1, figsize=(w, h))
        r = 0
        style = {**{s:"red" if "_Call" in s else "green" for s in strikes}, "NIFTY": "black"}
        plot_x_diff(oi_opt_df/1000, include_strikes=[strikes[s]], ax=axs[r], title="Change in OI - ROC", style=style, kind="bar")
        r = r + 1
        # plot_df(vol_opt_df.T[[strikes[s]]], ax=axs[r], title="VOL", style=style, kind="bar")
        plot_x_roc(oi_opt_df/1000, include_strikes=[strikes[s]], ax=axs[r], title="Change in OI", style=style, kind="bar")
        r = r + 1
        sr_arr = [price_opt_df.loc[strikes[s]].copy(), spot_df.T.copy()]
        price_opt_spot_df = pd.concat(sr_arr, axis=1)
        plot_df(price_opt_spot_df, plot_spot=True, ax=axs[r], title="Premium vs Spot", style=style)
        # plot_x_roc(price_opt_spot_df.T, include_strikes=[strikes[s]], ax=axs[r], title="Premium vs Spot", style=style, plot_spot=True)
        # plot_x_roc(price_opt_df, include_strikes=[strikes[s]], ax=axs[r], title="Premium", style=style)
        # plot_df(compute_money_flow_ohlcv([strikes[s]], ohlcv_map), ax=axs[r], title="Money Flow", style=style)
        # plot_x_roc(prem_decay_df, include_strikes=[strikes[s]], ax=axs[r], title="Premium Decay", style=style)
        if save_filepath:
            plt.savefig(f"{save_filepath}/{strikes[s]}.png", bbox_inches="tight")
            plt.close(fig)
#

def plot_individual_futures(expiries, save_filepath=None):
    ohlcv_map = {
        "O" : price_open_fut_df,
        "H" : price_high_fut_df,
        "L" : price_low_fut_df,
        "C" : price_fut_df,
        "V" : vol_fut_df,
    }
    nrows = 2
    w = 30
    h = 18
    for s in range(0, len(expiries)):
        fig, axs = plt.subplots(nrows=nrows, ncols=1, figsize=(w, h))
        r = 0
        style = {**{e:"blue" for e in expiries}, "NIFTY": "black"}
        plot_x_diff(oi_fut_df/1000, include_strikes=[expiries[s]], ax=axs[r], title="OI", style=style, kind="bar")
        # r = r + 1
        # plot_df(vol_fut_df.T[[expiries[s]]], ax=axs[r], title="VOL", style=style, kind="bar")
        r = r + 1
        sr_arr = [price_fut_df.loc[expiries[s]].copy(), spot_df.T.copy()]
        price_fut_spot_df = pd.concat(sr_arr, axis=1)
        plot_df(price_fut_spot_df, plot_spot=True, ax=axs[r], title="Premium vs Spot", style=style)
        # plot_x_roc(price_fut_spot_df.T, include_strikes=[expiries[s]], ax=axs[r], title="Premium vs Spot", style=style, plot_spot=True)
        # plot_x_roc(price_fut_df, include_strikes=[expiries[s]], ax=axs[r], title="Premium", style=style)
        # plot_df(compute_money_flow_ohlcv([expiries[s]], ohlcv_map), ax=axs[r], title="Money Flow", style=style)
        if save_filepath:
            plt.savefig(f"{save_filepath}/{expiries[s]}.png", bbox_inches="tight")
            plt.close(fig)

## Show Charts

In [ ]:
# if PLOT_MODE == "SHOW_CHARTS":
#     build_dataframes()

### Call

In [ ]:
# if PLOT_MODE == "SHOW_CHARTS":
#     call_strikes = sorted([s for s in OPT_STRIKES if "00_Call" in s], reverse=True)
#     plot_strikes(call_strikes)

### Put

In [ ]:
# if PLOT_MODE == "SHOW_CHARTS":
#     put_strikes = sorted([s for s in OPT_STRIKES if "00_Put" in s], reverse=False)
#     plot_strikes(put_strikes)

### Futures

In [ ]:
# if PLOT_MODE == "SHOW_CHARTS":
#     plot_futures(FUT_EXPIRIES)

## Save Charts

In [ ]:
!mkdir -p downloads

In [ ]:
!mkdir -p plots

In [ ]:
!rm -f downloads/*

In [ ]:
!rm -f plots/*

In [ ]:
if PLOT_MODE == "DOWNLOAD_LIVE_CHARTS":
    CHARTIQ_DATA_TODAY_PATH = "downloads"
    OPT_EXPIRY_DATE = WK_OPT_EXPIRY_DATE
    download_n_save_data()
    spot_min = int(spot_df.T["NIFTY"].min())
    spot_max = int(spot_df.T["NIFTY"].max())
    call_strikes = [int(s.replace("_Call", "")) for s in OPT_STRIKES if "00_Call" in s]
    call_strikes = [f"{s}_Call" for s in call_strikes if s >= spot_min - 200 and s <= spot_max + 100]
    call_strikes = sorted(call_strikes + [s for s in LIVE_CHARTS_FII_STRIKES if "00_Call" in s and s not in call_strikes], reverse=True)
    put_strikes = [int(s.replace("_Put", "")) for s in OPT_STRIKES if "00_Put" in s]
    put_strikes = [f"{s}_Put" for s in put_strikes if s >= spot_min - 100 and s <= spot_max + 200]
    put_strikes = sorted(put_strikes + [s for s in LIVE_CHARTS_FII_STRIKES if "00_Put" in s and s not in put_strikes], reverse=False)
    save_filepath = "plots"
    plot_individual_strikes(call_strikes, save_filepath)
    plot_individual_strikes(put_strikes, save_filepath)

In [ ]:
if PLOT_MODE == "DOWNLOAD_CHARTS":
    CHARTIQ_DATA_TODAY_PATH = f"{CHARTIQ_DATA_DIR}/{YYYY:04}/{MM:02}/{DD:02}"
    OPT_EXPIRY_DATE = WK_OPT_EXPIRY_DATE
    download_n_save_data()

In [ ]:
if PLOT_MODE == "PLOT_CHARTS":
    CHARTIQ_DATA_TODAY_PATH = f"{CHARTIQ_DATA_DIR}/{YYYY:04}/{MM:02}/{DD:02}"
    build_dataframes()
    spot_min = int(spot_df.T["NIFTY"].min())
    spot_max = int(spot_df.T["NIFTY"].max())
    call_strikes = sorted([int(s.replace("_Call", "")) for s in OPT_STRIKES if "00_Call" in s], reverse=True)
    call_strikes = sorted([f"{s}_Call" for s in call_strikes if s >= spot_min - 500 and s <= spot_max + 500], reverse=True)
    put_strikes = sorted([int(s.replace("_Put", "")) for s in OPT_STRIKES if "00_Put" in s], reverse=False)
    put_strikes = sorted([f"{s}_Put" for s in put_strikes if s >= spot_min - 500 and s <= spot_max + 500], reverse=False)
    save_filepath = "plots"
    plot_individual_strikes(call_strikes, save_filepath)
    plot_individual_strikes(put_strikes, save_filepath)
    plot_individual_futures(FUT_EXPIRIES, save_filepath)

In [ ]:
!rm -f plots.zip

In [ ]:
!zip -rq plots.zip plots

## Options OI / Moneyflow

In [ ]:
if PLOT_MODE == "OPT_LIVE":
    CHARTIQ_DATA_TODAY_PATH = f"{CHARTIQ_DATA_DIR}/{YYYY:04}/{MM:02}/{DD:02}"
    opt_df_map = download_options_live_data()
    oi_opt_df, vol_opt_df, price_opt_df, price_open_opt_df, price_high_opt_df, price_low_opt_df = build_fno_stk_df(opt_df_map)

if PLOT_MODE == "SHOW_CHARTS":
    CHARTIQ_DATA_TODAY_PATH = f"{CHARTIQ_DATA_DIR}/{YYYY:04}/{MM:02}/{DD:02}"
    build_dataframes()

In [ ]:
if PLOT_MODE == "OPT_LIVE":
    call_strikes = sorted([s for s in OPT_STRIKES if "00_Call" in s], reverse=True)
    put_strikes = sorted([s for s in OPT_STRIKES if "00_Put" in s], reverse=False)

if PLOT_MODE == "SHOW_CHARTS":
    call_strikes = ["24100_Call", "24300_Call", "24200_Call"] # decreasing order
    put_strikes = ["24200_Put", "24300_Put", "24400_Put"] # increasing order

In [ ]:
# # Intraday individual ATM strikes MoneyFlow
# strikes = call_strikes + put_strikes
# ohlcv_map = {
#     "O" : price_open_opt_df,
#     "H" : price_high_opt_df,
#     "L" : price_low_opt_df,
#     "C" : price_opt_df,
#     "V" : vol_opt_df,
# }
# style = {**{s:"red" if "_Call" in s else "green" for s in strikes}}
# nrows = 3
# ncols = 2
# fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20*ncols, 8*nrows))
# r = 0
# plot_df(compute_money_flow_ohlcv([call_strikes[r]], ohlcv_map), ax=axs[r, 0], title="Money Flow", style=style)
# plot_df(compute_money_flow_ohlcv([put_strikes[r]], ohlcv_map), ax=axs[r, 1], title="Money Flow", style=style)
# r = r + 1
# plot_df(compute_money_flow_ohlcv([call_strikes[r]], ohlcv_map), ax=axs[r, 0], title="Money Flow", style=style)
# plot_df(compute_money_flow_ohlcv([put_strikes[r]], ohlcv_map), ax=axs[r, 1], title="Money Flow", style=style)
# r = r + 1
# plot_df(compute_money_flow_ohlcv([call_strikes[r]], ohlcv_map), ax=axs[r, 0], title="Money Flow", style=style)
# plot_df(compute_money_flow_ohlcv([put_strikes[r]], ohlcv_map), ax=axs[r, 1], title="Money Flow", style=style)

In [ ]:
# def plot_atm_net_moneyflow(save_filepath=None):
#     ohlcv_map = {
#         "O" : price_open_opt_df,
#         "H" : price_high_opt_df,
#         "L" : price_low_opt_df,
#         "C" : price_opt_df,
#         "V" : vol_opt_df,
#     }
#     # call_strikes = sorted([s for s in OPT_STRIKES if "00_Call" in s], reverse=True)
#     # put_strikes = sorted([s for s in OPT_STRIKES if "00_Put" in s], reverse=False)
#     w = 18
#     h = 6
#     fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(w, h))
#     atm_mf_df = compute_money_flow_ohlcv(call_strikes + put_strikes, ohlcv_map)
#     # atm_mf_df = oi_opt_df.T[call_strikes + put_strikes].copy()
#     atm_net_mf_df = atm_mf_df[call_strikes].sum(axis=1) - atm_mf_df[put_strikes].sum(axis=1)
#     # atm_net_mf_df = atm_mf_df[put_strikes].sum(axis=1) - atm_mf_df[call_strikes].sum(axis=1)
#     atm_net_mf_df = atm_net_mf_df.to_frame()
#     atm_net_mf_df.columns = ["MF"]
#     atm_net_mf_df["NIFTY"] = spot_df.T
#     style = {"MF": "black"}
#     plot_df(atm_net_mf_df, ax=ax, title="ATM Money Flow", plot_spot=True, style=style)
#     if save_filepath:
#         plt.savefig(save_filepath, bbox_inches="tight")
#         plt.close(fig)
# #

# if PLOT_MODE in ("OPT_LIVE", "SHOW_CHARTS"):
#     plot_atm_net_moneyflow()

In [ ]:
# def plot_atm_net_oi(save_filepath=None):
#     atm_oi_df = oi_opt_df.T[call_strikes + put_strikes].copy()
#     atm_oi_df["Put"] = atm_oi_df[put_strikes].sum(axis=1)
#     atm_oi_df["Call"] = atm_oi_df[call_strikes].sum(axis=1)
#     atm_oi_df["NIFTY"] = spot_df.T
#     atm_oi_df = atm_oi_df.T
#     w = 18
#     h = 6
#     fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(w, h))
#     style = {"Put": "green", "Call": "red"}
#     plot_x_roc(atm_oi_df, include_strikes=["Call", "Put"], ax=ax, title="ATM Net OI Trend", plot_spot=True, style=style)
#     if save_filepath:
#         plt.savefig(save_filepath, bbox_inches="tight")
#         plt.close(fig)
# #

# if PLOT_MODE in ("OPT_LIVE", "SHOW_CHARTS"):
#     plot_atm_net_oi()

In [ ]:
# def plot_individual_atm_strikes_oi(save_filepath=None):
#     atm_oi_df = oi_opt_df.T[call_strikes + put_strikes].copy()
#     atm_oi_df["NIFTY"] = spot_df.T
#     atm_oi_df = atm_oi_df.T
#     w = 18
#     h = 6
#     fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(w, h))
#     style = {**{s:"red" for s in call_strikes}, **{s:"green" for s in put_strikes}, "NIFTY": "--k"}
#     plot_x_roc(atm_oi_df, include_strikes=(call_strikes + put_strikes), ax=ax, title="ATM OI Trend", plot_spot=True, style=style)
#     if save_filepath:
#         plt.savefig(save_filepath, bbox_inches="tight")
#         plt.close(fig)
# #

# if PLOT_MODE in ("OPT_LIVE", "SHOW_CHARTS"):
#     plot_individual_atm_strikes_oi()

In [ ]:
# def plot_atm_net_coi(save_filepath=None):
#     atm_oi_df = oi_opt_df.T[call_strikes + put_strikes].copy()
#     atm_net_coi_df = atm_oi_df[put_strikes].sum(axis=1) - atm_oi_df[call_strikes].sum(axis=1)
#     atm_net_coi_df = atm_net_coi_df.to_frame()
#     atm_net_coi_df.columns = ["COI"]
#     atm_net_coi_df["NIFTY"] = spot_df.T
#     w = 18
#     h = 6
#     fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(w, h))
#     plot_df(atm_net_coi_df, ax=ax, title="ATM NET COI", style={"COI": "orange"}, plot_spot=True, kind="bar")
#     if save_filepath:
#         plt.savefig(save_filepath, bbox_inches="tight")
#         plt.close(fig)
# #

# if PLOT_MODE in ("OPT_LIVE", "SHOW_CHARTS"):
#     plot_atm_net_coi()

## NIFTY Top 10 Moneyflow

In [ ]:
if PLOT_MODE == "NIFTY_LIVE":
    idx_df_map = download_index_data(NIFTY_INSTRUMENTS_DF)
    spot_df, _ = build_idx_df(idx_df_map)
    stk_df_map = download_nifty_top10_live_data()
    _, vol_stk_df, price_close_stk_df, price_open_stk_df, price_high_stk_df, price_low_stk_df = build_fno_stk_df(stk_df_map, oi=False)

In [ ]:
def plot_nifty_top10_moneyflow(save_filepath=None):
    nifty_top10 = NIFTY_TOP10_INSTRUMENTS.keys()
    w = 18
    h = 6
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(w, h))
    ohlcv_map = {
        "O" : price_open_stk_df,
        "H" : price_high_stk_df,
        "L" : price_low_stk_df,
        "C" : price_close_stk_df,
        "V" : vol_stk_df,
    }
    mf_df = compute_money_flow_ohlcv(nifty_top10, ohlcv_map)
    for symbol, instrument_map in NIFTY_TOP10_INSTRUMENTS.items():
        mf_df[symbol] = mf_df[symbol] * instrument_map["weightage"]
    net_mf_df = mf_df[nifty_top10].sum(axis=1)
    net_mf_df = net_mf_df.to_frame()
    net_mf_df.columns = ["MF"]
    net_mf_df["NIFTY"] = spot_df.T
    plot_df(net_mf_df, ax=ax, title="NIFTY Money Flow", style={"MF": "orange"}, plot_spot=True, kind="bar")
    if save_filepath:
        plt.savefig(save_filepath, bbox_inches="tight")
        plt.close(fig)
#

if PLOT_MODE == "NIFTY_LIVE":
    plot_nifty_top10_moneyflow()

##Call vs Put

In [ ]:
# call_strikes = sorted([s for s in OPT_STRIKES if "00_Call" in s], reverse=True)
# put_strikes = sorted([s for s in OPT_STRIKES if "00_Put" in s], reverse=False)
# nrows = 6
# ncols = 4
# fig, axs = plt.subplots(nrows=nrows, ncols=len(call_strikes), figsize=(10*ncols, 6*nrows))
# if len(call_strikes) == 1:
#     axs = axs.reshape(len(axs),1)
# for s in range(0, len(call_strikes)):
#     r = 0
#     style = {call_strikes[s]: "red", put_strikes[s]: "green", "NIFTY": "--k"}
#     plot_x_diff(oi_opt_df, include_strikes=[call_strikes[s], put_strikes[s]], ax=axs[r, s], title="OI", style=style)
#     r = r + 1
#     if history_api_data:
#         plot_df(vol_opt_df.T[[call_strikes[s], put_strikes[s]]], ax=axs[r, s], title="VOL", style=style)
#     else:
#         plot_x_diff(vol_opt_df, include_strikes=[call_strikes[s], put_strikes[s]], ax=axs[r, s], title="VOL", style=style)
#     r = r + 1
#     plot_x_roc(price_opt_df, include_strikes=[call_strikes[s], put_strikes[s]], ax=axs[r, s], title="Premium", style=style)
#     r = r + 1
#     plot_x_roc(iv_nr_df, include_strikes=[call_strikes[s], put_strikes[s]], ax=axs[r, s], title="IV by NR", style=style)
#     r = r + 1
#     plot_x_roc(prem_decay_df, include_strikes=[call_strikes[s], put_strikes[s]], ax=axs[r, s], title="Premium Decay", style=style)
#     r = r + 1
#     money_flow_df = None
#     if history_api_data:
#         money_flow_df = compute_money_flow_ohlcv([call_strikes[s], put_strikes[s]])
#     else:
#         money_flow_df = compute_money_flow_df([call_strikes[s], put_strikes[s]])
#     plot_df(money_flow_df, ax=axs[r, s], title="Money Flow", style=style)

##Call vs Call & Put vs Put

In [ ]:
# money_flow_df = None
# if history_api_data:
#     money_flow_df = compute_money_flow_ohlcv([s for s in OPT_STRIKES if "00_" in s]).T
# else:
#     money_flow_df = compute_money_flow_df([s for s in OPT_STRIKES if "00_" in s]).T

# nrows = 2
# ncols = 4
# fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10*ncols, 6*nrows))
# c = 0
# plot_x_roc(price_opt_df, include_strikes=[s for s in OPT_STRIKES if "00_Call" in s], ax=axs[0, c], title="Call Premium", style=COLORS_STYLE)
# plot_x_roc(price_opt_df, include_strikes=[s for s in OPT_STRIKES if "00_Put" in s], ax=axs[1, c], title="Put Premium", style=COLORS_STYLE)
# c = c + 1
# plot_x_roc(iv_nr_df, include_strikes=[s for s in OPT_STRIKES if "00_Call" in s], ax=axs[0, c], title="Call IV", style=COLORS_STYLE)
# plot_x_roc(iv_nr_df, include_strikes=[s for s in OPT_STRIKES if "00_Put" in s], ax=axs[1, c], title="Put IV", style=COLORS_STYLE)
# c = c + 1
# plot_x_roc(prem_decay_df, include_strikes=[s for s in OPT_STRIKES if "00_Call" in s], ax=axs[0, c], title="Call Premium Decay", style=COLORS_STYLE)
# plot_x_roc(prem_decay_df, include_strikes=[s for s in OPT_STRIKES if "00_Put" in s], ax=axs[1, c], title="Put Premium Decay", style=COLORS_STYLE)
# c = c + 1
# plot_x_roc(money_flow_df, include_strikes=[s for s in OPT_STRIKES if "00_Call" in s], ax=axs[0, c], title="Call Moneyflow", style=COLORS_STYLE)
# plot_x_roc(money_flow_df, include_strikes=[s for s in OPT_STRIKES if "00_Put" in s], ax=axs[1, c], title="Put Moneyflow", style=COLORS_STYLE)

In [ ]:
# normalized_df = get_normalized_n_consolidate_call_n_put(price_opt_df)

# style = {"Call": "red", "Put": "green", "NIFTY": "--k"}
# fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(16, 4*1))
# plot_df(normalized_df[["Call"]], ax=axs[0], title="Call Normalized Premium", style=style)
# plot_df(normalized_df[["Put"]], ax=axs[1], title="Put Normalized Premium", style=style)

In [ ]:
# fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(16, 6*2))
# plot_x_roc(price_opt_df, include_strikes=[s for s in OPT_STRIKES if "00_Call" in s], ax=axs[0], title="Call Premium", style=COLORS_STYLE)
# plot_x_roc(price_opt_df, include_strikes=[s for s in OPT_STRIKES if "00_Put" in s], ax=axs[1], title="Put Premium", style=COLORS_STYLE)

In [ ]:
# fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(16, 6*2))
# plot_x_roc(iv_nr_df, include_strikes=[s for s in OPT_STRIKES if "00_Call" in s], ax=axs[0], title="Call IV by NR", style=COLORS_STYLE)
# plot_x_roc(iv_nr_df, include_strikes=[s for s in OPT_STRIKES if "00_Put" in s], ax=axs[1], title="Put IV by NR", style=COLORS_STYLE)

In [ ]:
# fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(16, 6*2))
# plot_x_roc(prem_decay_df, include_strikes=[s for s in OPT_STRIKES if "00_Call" in s], ax=axs[0], title="Call Premium Decay", style=COLORS_STYLE)
# plot_x_roc(prem_decay_df, include_strikes=[s for s in OPT_STRIKES if "00_Put" in s], ax=axs[1], title="Put Premium Decay", style=COLORS_STYLE)

# Misc Scripts



```
Reminder Scripts
----------------

MAC
---
while true; do osascript -e 'display alert "15 min reminder"'; ts=$(date +%s); remain_secs=$((900-$ts%900)); sleep $remain_secs; done;
while true; do osascript -e 'display notification "15 min reminder" with title "15 min reminder"'; sleep 900; done;
while true; do osascript -e 'display notification "15 min reminder" with title "15 min reminder"'; ts=$(date +%s); remain_secs=$((900-$ts%900)); sleep $remain_secs; done;

Windows
-------
15-mins-reminder.bat
--------------------
@echo off
:loop
msg * "15 mins reminder"
timeout /t 900
goto loop
---------------------
@echo off
:loop
@set /a "run_every_n_mins = 15"
msg * "%run_every_n_mins% mins reminder"
@set /a "n_secs = run_every_n_mins * 60"
@set /a "mins = %time:~3,2%"
@set /a "secs = %time:~6,2%"
@set /a "ts = mins * 60 + secs"
@set /a "remain_secs = n_secs - (ts %% n_secs)"
timeout /t %remain_secs%
goto loop
--------------------

```





```
Excel formula to filter current expiry 100 multiple strikes
-----------------------------------------------------------
=AND(A2="OPTIDX", B2="NIFTY", C2=DATE(2024,1,11), MOD(D2,100)=0)
```

